# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6fd29d7400>
├── 'a' --> tensor([[-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824]])
└── 'x' --> <FastTreeValue 0x7f6fd2a3ae20>
    └── 'c' --> tensor([[ 0.7229, -0.7421,  1.7529,  1.1602],
                        [ 0.2453, -0.0814,  0.6416, -1.5886],
                        [-0.0930, -0.2499, -0.3876,  0.2008]])

In [4]:
t.a

tensor([[-1.2561,  1.6080, -0.2397],
        [ 0.2446,  1.6167,  1.0824]])

In [5]:
%timeit t.a

71.7 ns ± 0.0964 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6fd29d7400>
├── 'a' --> tensor([[-1.5874,  1.5187,  0.7893],
│                   [-1.1386,  0.5283, -0.1752]])
└── 'x' --> <FastTreeValue 0x7f6fd2a3ae20>
    └── 'c' --> tensor([[ 0.7229, -0.7421,  1.7529,  1.1602],
                        [ 0.2453, -0.0814,  0.6416, -1.5886],
                        [-0.0930, -0.2499, -0.3876,  0.2008]])

In [7]:
%timeit t.a = new_value

71.6 ns ± 0.041 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824]]),
    x: Batch(
           c: tensor([[ 0.7229, -0.7421,  1.7529,  1.1602],
                      [ 0.2453, -0.0814,  0.6416, -1.5886],
                      [-0.0930, -0.2499, -0.3876,  0.2008]]),
       ),
)

In [10]:
b.a

tensor([[-1.2561,  1.6080, -0.2397],
        [ 0.2446,  1.6167,  1.0824]])

In [11]:
%timeit b.a

64.4 ns ± 0.0946 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0095, -0.2527,  0.0777],
               [-0.7498, -1.6789,  0.6341]]),
    x: Batch(
           c: tensor([[ 0.7229, -0.7421,  1.7529,  1.1602],
                      [ 0.2453, -0.0814,  0.6416, -1.5886],
                      [-0.0930, -0.2499, -0.3876,  0.2008]]),
       ),
)

In [13]:
%timeit b.a = new_value

551 ns ± 2.22 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

891 ns ± 1.53 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 934 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6ef82a9970>
├── 'a' --> tensor([[[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]],
│           
│                   [[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]],
│           
│                   [[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]],
│           
│                   [[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]],
│           
│                   [[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]],
│           
│                   [[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]],
│           
│                   [[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]],
│           
│                   [[-1.2561,  1.6080, -0.2397],
│                    [ 0.2446,  1.6167,  1.0824]]])
└── 'x' --> <FastTreeValue 0x7f6fd2a52a90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 29.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6f001e29a0>
├── 'a' --> tensor([[-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824],
│                   [-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824],
│                   [-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824],
│                   [-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824],
│                   [-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824],
│                   [-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824],
│                   [-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824],
│                   [-1.2561,  1.6080, -0.2397],
│                   [ 0.2446,  1.6167,  1.0824]])
└── 'x' --> <FastTreeValue 0x7f6f001e2fa0>
    └── 'c' --> tensor([[ 0.7229, -0.7421,  1.7529,  1.1602],
                        [ 0.2453, -0.0814,  0.6416, -1.5886],
                 

In [23]:
%timeit t_cat(trees)

31.4 µs ± 50.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68 µs ± 597 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]],
       
               [[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]],
       
               [[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]],
       
               [[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]],
       
               [[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]],
       
               [[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]],
       
               [[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]],
       
               [[-1.2561,  1.6080, -0.2397],
                [ 0.2446,  1.6167,  1.0824]]]),
    x: Batch(
           c: tensor([[[ 0.7229, -0.7421,  1.7529,  1.1602],
                       [ 0.2453, -0.0814,  0.6416, -1.5886],
                       [-0.0930, -0.2499, -0.3876,  0.2008]],
         

In [26]:
%timeit Batch.stack(batches)

83.7 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824],
               [-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824],
               [-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824],
               [-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824],
               [-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824],
               [-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824],
               [-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824],
               [-1.2561,  1.6080, -0.2397],
               [ 0.2446,  1.6167,  1.0824]]),
    x: Batch(
           c: tensor([[ 0.7229, -0.7421,  1.7529,  1.1602],
                      [ 0.2453, -0.0814,  0.6416, -1.5886],
                      [-0.0930, -0.2499, -0.3876,  0.2008],
                      [ 0.7229, -0.7421,  1.7529,  1.1602],
                      [ 0.2453, -

In [28]:
%timeit Batch.cat(batches)

177 µs ± 470 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

345 µs ± 5.58 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
